In [1]:
import spacy
import pandas as pd
from spacy import displacy

In [2]:
# usa jobs file
file = './Data/Cleaned Data/USAJobs.csv'

In [3]:
# read csv and create single desc column
df = pd.read_csv(file)

In [4]:
df['desc'] = df['Duties'] + df['Qualifications']

In [5]:
len(df)

11892

In [6]:
df.head()

,Title,GS_Low,GS_High,Department,Organization,Duties,Qualifications,desc
0,Information Technology Speciaist (INFOSEC),13,13,Department Of Health And Human Services,Office of the Secretary of Health and Human Se...,• Serves as a technical expert and performs a ...,Specialized Experience: Qualifying experience...,• Serves as a technical expert and performs a ...
1,Information Technology Specialist (InfoSec),14,14,NaN,U.S. International Trade Commission,As an Information Technology Specialist (INFOS...,You must meet all the requirements below by t...,As an Information Technology Specialist (INFOS...
2,IT SPECIALIST (INFOSEC),12,12,Department of Defense,Defense Finance and Accounting Service,"As the Information Assurance Manager (IAM), im...",One year of specialized experience equivalent ...,"As the Information Assurance Manager (IAM), im..."
3,IT SPECIALIST (INFOSEC),12,12,Department of Defense,Defense Finance and Accounting Service,"Develops, plans, and implements the Automated ...",Basic Requirement:Applicants must have IT-rela...,"Develops, plans, and implements the Automated ..."
4,Information Technology Specialist (InfoSec),14,14,NaN,U.S. International Trade Commission,As an Information Technology Specialist (INFOS...,You must meet all the requirements below by t...,As an Information Technology Specialist (INFOS...


In [7]:
# subset to sample for easier testing
all_desc = df['desc'][0:10].tolist()

In [8]:
# loads spacy pre-trained model
# nlp = spacy.load('en_core_web_lg')

In [9]:
# scanes all_desc list 
# scanned = list(nlp.pipe(all_desc))

In [70]:
# example baseline cybersecurity words of skills
baseline =['cybersecurity', 'security', 'cyber', 'adversary model', 'attacks', 'intrusion', 'password','hacking', 'cracking'
          'backdoor', 'trojan', 'virus', 'wireless attacks', 'sniffing', 'spoofing', 'denial of service', 'dos', 'bot',
           'mac', 'wb app', '0-day exploits', 'vulnerabilities', 'social engineering', 'cert', 'ids', 'detection', 'network',
           'information technology', 'gs-13', 'network security'
          ]

In [11]:
# scans the baseline 
base_scan = nlp(baseline)

In [12]:
# matched = []
# similar = []
# missing = []
# for token in base_scan:
#     if token.is_stop != True and token.is_punct != True and len(token.text) > 2 and token.has_vector:
#         for t2 in scanned[0]:
#             if t2.has_vector:
#                 sim_score = token.similarity(t2)
#                 if sim_score > 0.9:
#                     matched.append(token)
#                     print(token.text, "<->", t2.text, token.similarity(t2))
#                 elif sim_score < 0.9 and sim_score >=0.7:
#                     similar.append(token)
#                 else:
#                     missing.append(token)

In [13]:
# matched = list(set(matched))
# similar = list(set(similar))
# missing = list(set(missing))

In [14]:
# total = len(matched) + len(similar) + len(missing)
# total

In [15]:
# matched_perc = len(matched) / total
# similar_perc = len(similar) / total
# missing_perc = len(missing) / total
# matched_perc, similar_perc, missing_perc

In [79]:
import spacy
from spacy.matcher import PhraseMatcher, Matcher
from spacy.tokens import Span
from spacy.pipeline import merge_entities
from spacy.language import Language

In [17]:
# attempts to create pipeline matcher like in Spacy2 but they changed a lot and its not working right
@Language.component('skills_pipe')
class TargetSkillsPipeline():
    name = "skills_pipe"
    def __init__(self, nlp):
        self.skills = ['cybersecurity', 'security', 'cyber', 'adversary model', 'attacks', 'intrusion', 'password','hacking', 'cracking'
          'backdoor', 'trojan', 'virus', 'wireless attacks', 'sniffing', 'spoofing', 'denial of service', 'dos', 'bot',
           'mac', 'wb app', '0-day exploits', 'vulnerabilities', 'social engineering', 'cert', 'ids', 'detection', 'network'
          ]
        
        self.label_hash = nlp.vocab.strings['SKILL']
        self.matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
        patterns = [nlp(word) for word in self.skills]
        self.matcher.add('TerminologyList', self.add_event_ent, *patterns)
        
    @staticmethod
    def add_event_ent(_, doc, i, matches):
        _, start, end = matches[i]
        entity = Span(doc, start, end, label='SKILL')
        doc.ents = tuple([e for e in doc.ents if not (e.start < end and e.end > start)]) + (entity,)
        
    def __call__(self, doc):
        matches = self.matcher(doc)
        for _, start, end in matches:
            span = doc[start:end]
        return doc


In [71]:
# instead create patters like so and feed into phrase matcher matcher spacy func
patterns = [[{"TEXT": word}] for word in baseline]
patterns

[[{'TEXT': 'cybersecurity'}],
 [{'TEXT': 'security'}],
 [{'TEXT': 'cyber'}],
 [{'TEXT': 'adversary model'}],
 [{'TEXT': 'attacks'}],
 [{'TEXT': 'intrusion'}],
 [{'TEXT': 'password'}],
 [{'TEXT': 'hacking'}],
 [{'TEXT': 'crackingbackdoor'}],
 [{'TEXT': 'trojan'}],
 [{'TEXT': 'virus'}],
 [{'TEXT': 'wireless attacks'}],
 [{'TEXT': 'sniffing'}],
 [{'TEXT': 'spoofing'}],
 [{'TEXT': 'denial of service'}],
 [{'TEXT': 'dos'}],
 [{'TEXT': 'bot'}],
 [{'TEXT': 'mac'}],
 [{'TEXT': 'wb app'}],
 [{'TEXT': '0-day exploits'}],
 [{'TEXT': 'vulnerabilities'}],
 [{'TEXT': 'social engineering'}],
 [{'TEXT': 'cert'}],
 [{'TEXT': 'ids'}],
 [{'TEXT': 'detection'}],
 [{'TEXT': 'network'}],
 [{'TEXT': 'information technology'}],
 [{'TEXT': 'gs-13'}],
 [{'TEXT': 'network security'}]]

In [74]:
# callback to do on match
def on_match(_, doc, i, matches):
    _, start, end = matches[i]
    entity = Span(doc, start, end, label='SKILL')
    print(entity)
    doc.ents = tuple([e for e in doc.ents if not (e.start < end and e.end > start)]) + (entity,)


In [75]:
# load pre-trained model and add the matcher patterns to the matcher
nlp = spacy.load('en_core_web_lg')
# nlp.add_pipe('skills_pipe', after="ner")
# nlp.add_pipe('merge_entities')
matcher = Matcher(nlp.vocab)
# patterns = [[{"TEXT": word} for word in baseline]]
matcher.add("SKILLS", patterns, on_match=on_match)

In [76]:
# scan all_dsc through model
scanned = list(nlp.pipe(all_desc))

In [77]:
# get the matches that matched the word in the doc
doc = scanned[1]

matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text, span.has_vector, span.label_)

security
security
vulnerabilities
security
security
security
network
vulnerabilities
network
security
security
16498319728200408761 SKILLS 15 16 security True 
16498319728200408761 SKILLS 30 31 security True 
16498319728200408761 SKILLS 33 34 vulnerabilities True 
16498319728200408761 SKILLS 57 58 security True 
16498319728200408761 SKILLS 141 142 security True 
16498319728200408761 SKILLS 166 167 security True 
16498319728200408761 SKILLS 598 599 network True 
16498319728200408761 SKILLS 603 604 vulnerabilities True 
16498319728200408761 SKILLS 612 613 network True 
16498319728200408761 SKILLS 629 630 security True 
16498319728200408761 SKILLS 641 642 security True 


In [78]:
scanned[1]

As an Information Technology Specialist (INFOSEC), you will: serves as a security operations center (SOC) analyst, and technical advisor on matters relating to security operations, vulnerabilities and threats to ITC computer and communication systems; participate in the planning, developing, implementing, and coordination of the information security program at USITC; analyze Executive Orders, legislation, and policy and guidance from OMB, NIST, as well as other Federal IT agencies for impact and/or implication on USITC networks; assess procedures and applications of information systems hardware, system software, overall system design, data communication, and capabilities of the systems; serve as an USITC technical consultant and assists in formulating agency directives, policy and guidance; develop papers and gives formal presentations on security problems, advancements and proposed solutions as a technical consultant in the IT field; implement vendor-supplied authentication, auditing 